- 01_matmul
- 02_fully_connected
- 02a_why_sqrt5
- 02b_initialization
- 03_minibatch_training

Table of contents
1. Initial Setup
2. Basic training Loop
3. Using parameters and optim
4. Dataset and DataLoader
5. Validation

# 1. Initial Setup

## Data

### Fastai for Colab env

In [ ]:
!git clone http://github.com/fastai/course-v3.git

In [2]:
%cd course-v3/nbs/dl2

/content/course-v3/nbs/dl2


In [13]:
from exp.nb_02 import *

In [15]:
!cat exp/nb_02.py


#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev_nb/02_fully_connected.ipynb

from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

from torch.nn import init

def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

from torch import nn

### import data

In [7]:
train_x, train_y, valid_x, valid_y = get_data()

In [8]:
train_x.shape, train_x.type()

(torch.Size([50000, 784]), 'torch.FloatTensor')

In [16]:
n_in, nh, n_out = train_x.shape[1], 32, train_y.max().item()+1; n_in, nh, n_out

(784, 32, 10)

[^1]

In [70]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x
    # def forward(self, x, y):
    #     for l in self.layers: x = l(x)
    #     return(self.loss(x, y))

In [72]:
m = Model(n_in, nh, n_out)

In [74]:
y_hat = m(train_x)

## Loss Function

### Softmax

* formula of softmax : $f(x) = \frac{exp(x)}{\sum exp(x_i)}$
* We need softmax at last layer since we have to map it to the probabilistic space
* code of softmax is below

In [63]:
def softmax(x): return x.exp() / x.exp().sum(-1, keepdim=True)
# def softmax(x): return x.exp() / x.exp().sum()

[^2]

In [75]:
p = softmax(y_hat); p[0].sum()

tensor(1., grad_fn=<SumBackward0>)

- I need *log* of softmax because we will use cross-entropy which has form of $H(X, q) = H(X) + D(p|q) = - \sum_{x} p(x)\  log\ q(x)$ [^3]

In [85]:
def log_softmax(x): return x.exp() / x.exp().sum(-1, keepdim=True)

In [87]:
a = log_softmax(y_hat)

### Cross Entropy

- see the footnote 3 regarding further information of cross entropy
- One possible reason is because an information theory is based on descrete and determinate value

### Negative log likelihood function

In [77]:
def nll(x, y): return -x[range(x.shape[0]), y].mean() #x: prediction, y:target 

In [78]:
nll(y_hat, train_y)

tensor(0.0161, grad_fn=<NegBackward>)

- Negative value is related to entropy, roughly variable which is rare is meaningful [^4]

### LogSumExp

- Further tasks (after reading sum refs) [^5]

In [82]:
def log_softmax(x): return x - x.logsumexp(-1, keepdim=True)

In [89]:
test_near(log_softmax(y_hat), a)

# 2. Basic Training Loop

~ code from here ~

# Notes

[^1]: I don't have to define parameters no more, but obviously at first I tried to iniailized Model with `w1, b1, ....`, meaning I didn't practice enough the previous part, part of the torch nn, where I get parameters

[^2]: be cautious since `train_x.sum(-1)` will squeeze the rank, so that you need arg `keepdim:bool`, false as default

[^3]: see *eq (2.46)*, Foundations of Natural Language Processing, Christopher D. Manning and Hinrich Schütze

[^4]: Should check for `cross-entropy $\approx$ softmax of negative likelihood`

[^5]: check later